In [101]:
import kagglehub
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [102]:
path = kagglehub.dataset_download("datamunge/sign-language-mnist")

In [103]:
print("Path to dataset files:", path)

Path to dataset files: /root/.cache/kagglehub/datasets/datamunge/sign-language-mnist/versions/1


In [104]:
train_data = pd.read_csv(f"{path}/sign_mnist_train.csv")
test_data = pd.read_csv(f"{path}/sign_mnist_test.csv")

In [105]:
# Separate labels and features
x_train = train_data.iloc[:, 1:].values / 255.0
y_train = train_data.iloc[:, 0].values
x_test = test_data.iloc[:, 1:].values / 255.0
y_test = test_data.iloc[:, 0].values

In [106]:
# Reshape data for CNN
x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)

In [107]:
# Define data augmentation generator
data_augmentation = ImageDataGenerator(
    rotation_range=15,      # Randomly rotate images by 15 degrees
    width_shift_range=0.1,  # Randomly shift images horizontally by 10%
    height_shift_range=0.1, # Randomly shift images vertically by 10%
    zoom_range=0.1,         # Randomly zoom images
    shear_range=0.1,        # Randomly apply shear transformations
    horizontal_flip=False,  # Flip images horizontally (not applicable for sign language)
)

In [108]:
train_data = data_augmentation.flow(x_train, y_train, batch_size=32)
test_data = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

In [109]:
# Build CNN model
model = Sequential([
    Input(shape=(28, 28, 1)),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.3),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(25, activation='softmax')  # Assuming 25 classes
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [110]:
# Train the model
history = model.fit(train_data, epochs=10, batch_size=32, validation_data=test_data)

# Evaluate the model
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f"Test accuracy: {test_acc:.2f}")


Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


858/858 ━━━━━━━━━━━━━━━━━━━━ 23s 21ms/step - accuracy: 0.2131 - loss: 2.5671 - val_accuracy: 0.7814 - val_loss: 0.5867
Epoch 2/10
858/858 ━━━━━━━━━━━━━━━━━━━━ 33s 15ms/step - accuracy: 0.6728 - loss: 0.9464 - val_accuracy: 0.9145 - val_loss: 0.2685
Epoch 3/10
858/858 ━━━━━━━━━━━━━━━━━━━━ 13s 16ms/step - accuracy: 0.7977 - loss: 0.5904 - val_accuracy: 0.9363 - val_loss: 0.1653
Epoch 4/10
858/858 ━━━━━━━━━━━━━━━━━━━━ 14s 16ms/step - accuracy: 0.8456 - loss: 0.4578 - val_accuracy: 0.9604 - val_loss: 0.1117
Epoch 5/10
858/858 ━━━━━━━━━━━━━━━━━━━━ 20s 16ms/step - accuracy: 0.8759 - loss: 0.3568 - val_accuracy: 0.9703 - val_loss: 0.0851
Epoch 6/10
858/858 ━━━━━━━━━━━━━━━━━━━━ 13s 16ms/step - accuracy: 0.9014 - loss: 0.2852 - val_accuracy: 0.9815 - val_loss: 0.0602
Epoch 7/10
858/858 ━━━━━━━━━━━━━━━━━━━━ 13s 15ms/step - accuracy: 0.9161 - loss: 0.2465 - val_accuracy: 0.9822 - val_loss: 0.0458
Epoch 8/10
858/858 ━━━━━━━━━━━━━━━━━━━━ 20s 15ms/step - accuracy: 0.9243 - loss: 0.2238 - val_accurac